In [6]:
#Import necessery libraries
import pandas as pd
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Download NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# Load the dataset
df = pd.read_csv('./train.csv')

In [8]:
# Data Preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.strip()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

df['clean_text'] = df['selected_text'].fillna('No text').apply(preprocess_text)

# Tokenization and Padding
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

# Label Encoding
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])
y = df['sentiment']

In [9]:
# Train-Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

# Build the RNN/LSTM model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=64, input_length=100))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 64)           640000    
                                                                 
 bidirectional_2 (Bidirecti  (None, 100, 128)          66048     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 100, 128)          0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                

In [11]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')


Epoch 1/10
687/687 [==============================] - 33s 38ms/step - loss: 0.6752 - accuracy: 0.7216 - val_loss: 0.5014 - val_accuracy: 0.8115
Epoch 2/10
687/687 [==============================] - 15s 22ms/step - loss: 0.4540 - accuracy: 0.8426 - val_loss: 0.4941 - val_accuracy: 0.8179
Epoch 3/10
687/687 [==============================] - 15s 21ms/step - loss: 0.3601 - accuracy: 0.8787 - val_loss: 0.5218 - val_accuracy: 0.8155
Epoch 4/10
687/687 [==============================] - 15s 21ms/step - loss: 0.3081 - accuracy: 0.8984 - val_loss: 0.5457 - val_accuracy: 0.8083
Epoch 5/10
687/687 [==============================] - 14s 21ms/step - loss: 0.2721 - accuracy: 0.9106 - val_loss: 0.5875 - val_accuracy: 0.7983
Epoch 6/10
687/687 [==============================] - 15s 22ms/step - loss: 0.2494 - accuracy: 0.9201 - val_loss: 0.6386 - val_accuracy: 0.8017
Epoch 7/10
687/687 [==============================] - 14s 21ms/step - loss: 0.2287 - accuracy: 0.9262 - val_loss: 0.6828 - val_accuracy:

In [12]:
# Save the model
model.save('sentiment_model.h5')

# Save the tokenizer
import joblib
joblib.dump(tokenizer, 'tokenizer.pkl')

# Save the label encoder
joblib.dump(label_encoder, 'label_encoder.pkl')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['label_encoder.pkl']